<center><img src='https://upload.wikimedia.org/wikipedia/fr/7/78/Logo_nouveau_CentraleSupelecParisSaclay.jpg' width=200></center>

<h6><center>M.Sc. DSBA</center></h6>
<h3><center>Deep learning Kaggle Competition</center></h3>


<h1>
<hr style=" border:none; height:3px;">
<center>Segment features around residential buildings in UAV images of flooded area taken in Houston after Hurrican Harvey.</center>
<hr style=" border:none; height:3px;">
</h1>


# Install Packages and load libraries


Specific versions of some packages were needed for the model to run

In [ ]:
! pip install utils
!pip install segmentation_models_pytorch
!pip install opencv-python-headless==4.1.2.30
!pip install --force-reinstall numpy==1.18.5
!pip install --force-reinstall folium==0.2.1
!pip install --force-reinstall Jinja2==2.10.1
!pip install -U albumentations>=0.3.0 --user
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install --upgrade albumentations
!pip install --upgrade git+https://github.com/albumentations-team/albumentations
!pip install --force-reinstall albumentations==1.0.3

Now restart the runtime

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

import os
import random

from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import datasets, transforms
from utils import *
import albumentations as album
import segmentation_models_pytorch as smp
sns.set(style='white', context='notebook', palette='deep')

# Data Preprocessing

In [ ]:
def keep_image_size_open(path, size=(512, 512)):
    img = Image.open(path)
    side = max(img.size)  # Get the longest side of the image
    mask = Image.new('RGB', (side, side), (0, 0, 0))  # Create a square canvas
    mask.paste(img, (0, 0))  # Paste the original image on the left top of the canvas
    mask = mask.resize(size)  # Resize the new image to a uniform size
    return mask

def keep_mask_size_open(path, size=(512, 512)):
    img = Image.open(path)
    side = max(img.size)  # Get the longest side of the image
    mask = Image.new('L', (side, side), 0)  # Create a square canvas
    mask.paste(img, (0, 0))  # Paste the original image on the left top of the canvas
    mask = mask.resize(size)  # Resize the new image to a uniform size
    return mask

In [ ]:
# helper function for data visualization
def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]); 
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

# Perform one hot encoding on label
def one_hot_encode(image,n_classes):
    x = F.one_hot(image,n_classes)
    return x
     
# Perform reverse one-hot-encoding on labels / preds
def reverse_one_hot(image):
    x = np.argmax(image, axis = -1)
    return x

# Perform colour coding on the reverse-one-hot outputs
def colour_code_segmentation(image, label_values):
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x

In [ ]:
def get_training_augmentation():
    train_transform = [
        album.PadIfNeeded(min_height=512, min_width=512, always_apply=True, border_mode=0),
        album.OneOf([album.HorizontalFlip(p=1),album.VerticalFlip(p=1),album.RandomRotate90(p=1),],p=0.5),
        album.ShiftScaleRotate(scale_limit=0.5,rotate_limit=0,shift_limit=0.1,p=0.5,border_mode=0), 
        album.GridDistortion(p=0.5)
    ]
    return album.Compose(train_transform)


def get_validation_augmentation():
    # Add sufficient padding to ensure image is divisible by 32
    test_transform = [
        album.PadIfNeeded(min_height=512, min_width=512, always_apply=True, border_mode=0),
    ]
    return album.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2,0,1).astype('float32')


def get_preprocessing(preprocessing_fn=None):  
    _transform = [
        album.Lambda(image=preprocessing_fn),
        album.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return album.Compose(_transform)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

class BackgroundDataset(torch.utils.data.Dataset):
    def __init__(
            self,path1, path2, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.path1 = path1
        self.path2 = path2
        self.name = os.listdir(os.path.join(path2, 'train_masks'))
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, idx):
        
        # read images and masks
        mask_name = self.name[idx]
        mask_path = os.path.join(self.path2,'train_masks',mask_name)
        img_path = os.path.join(self.path1,'train_images',mask_name.replace('png','jpg'))

        
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(cv2.imread(mask_path),0)
        image = keep_image_size_open(img_path)
        mask = keep_mask_size_open(mask_path)
        
        image = np.asarray(image).astype('int64')
        mask = np.asarray(mask).astype('int64')
        
        # one-hot-encode the mask  
        mask = torch.from_numpy(mask).to(torch.int64)
        mask = one_hot_encode(mask,27)
               
         #Augmentation
        mask = np.asarray(mask).astype('int64')
        sample = self.augmentation(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']
        
      
         # preprocessing applied only on numpy array image
        sample = self.preprocessing(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']
                  
        return image,mask
        
    def __len__(self):
        # return length of 
        return len(self.name)

# DataLoaders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if __name__ == '__main__':
    data = BackgroundDataset('/content/drive/MyDrive/fdl21-fdl-dsba/train_images',
                             '/content/drive/MyDrive/fdl21-fdl-dsba/train_masks',
                             augmentation=get_training_augmentation(),preprocessing=get_preprocessing(preprocessing_fn))
    check_image = data[35][0] # checking for the random 35th image
    check_mask = data[10|0][1]
    print(check_image.shape,check_mask.shape)
    print(check_image.dtype,check_mask.dtype)
    print(len(data))

(3, 512, 512) (27, 512, 512)
float32 float32
261


In [ ]:
batch_size = 10
num_workers = 4
# Splitting into Train and Val
full_dataset = BackgroundDataset('/content/drive/MyDrive/fdl21-fdl-dsba/train_images',
                             '/content/drive/MyDrive/fdl21-fdl-dsba/train_masks',
                             augmentation=get_training_augmentation(),preprocessing=get_preprocessing(preprocessing_fn))
train_size = int(0.9 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Creating  data_loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,num_workers=num_workers,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,num_workers=num_workers,shuffle=False)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
it, lt = next(iter(train_loader))
print(it.shape,lt.shape)
print(it.dtype,lt.dtype)

print(len(train_loader)*batch_size,len(val_loader)*batch_size)

# Model set

In [ ]:
# Need to be run only one time
ENCODER = 'resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = 27
ACTIVATION = 'softmax2d' #'softmax2d' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.PSPNet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=27,  
    activation=ACTIVATION
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
# Set flag to train the model or not. If set to 'False', only prediction is performed (using an older model checkpoint)
TRAINING = True

# Set num of epochs
EPOCHS = 20

# Set device: `cuda` or `cpu`
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define loss function
loss = smp.utils.losses.CrossEntropyLoss()

# define metrics
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]


optimizer = torch.optim.SGD([ 
    dict(params=model.parameters(), lr=0.003),
])
# define learning rate scheduler (not used in this NB)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=1, T_mult=2, eta_min=5e-5,
)

if os.path.exists('../input/pyramid-scene-parsing-pspnet-resnext50-pytorch/best_model.pth'):
    model = torch.load('../input/pyramid-scene-parsing-pspnet-resnext50-pytorch/best_model.pth', map_location=DEVICE)

# Model Training

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
%%time

if TRAINING:

    best_iou_score = 0.0
    train_logs_list, valid_logs_list = [], []

    for i in range(0, EPOCHS):

        # Perform training & validation
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(val_loader)
        train_logs_list.append(train_logs)
        valid_logs_list.append(valid_logs)

        # Save model if a better val IoU score is obtained
        if best_iou_score < valid_logs['iou_score']:
            best_iou_score = valid_logs['iou_score']
            torch.save(model, '/content/drive/MyDrive/fdl21-fdl-dsba/best_model_psnet3.pt')
            print('Model saved!')




Epoch: 0
train:   0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


valid: 100%|██████████| 3/3 [00:45<00:00, 15.22s/it, cross_entropy_loss - 3.298, iou_score - 4.36e-14]
Model saved!

Epoch: 1
valid: 100%|██████████| 3/3 [00:46<00:00, 15.65s/it, cross_entropy_loss - 3.245, iou_score - 0.03187]
Model saved!

Epoch: 2
valid: 100%|██████████| 3/3 [00:45<00:00, 15.21s/it, cross_entropy_loss - 3.15, iou_score - 0.1451]
Model saved!

Epoch: 3
valid: 100%|██████████| 3/3 [00:47<00:00, 15.85s/it, cross_entropy_loss - 3.015, iou_score - 0.2397]
Model saved!

Epoch: 4
valid: 100%|██████████| 3/3 [00:45<00:00, 15.16s/it, cross_entropy_loss - 2.91, iou_score - 0.3287]
Model saved!

Epoch: 5
valid: 100%|██████████| 3/3 [00:48<00:00, 16.16s/it, cross_entropy_loss - 2.889, iou_score - 0.3366]
Model saved!

Epoch: 6
valid: 100%|██████████| 3/3 [00:46<00:00, 15.48s/it, cross_entropy_loss - 2.875, iou_score - 0.3407]
Model saved!

Epoch: 7
valid: 100%|██████████| 3/3 [00:48<00:00, 16.07s/it, cross_entropy_loss - 2.868, iou_score - 0.344]
Model saved!

Epoch: 8
valid: 1

In [ ]:
PATH = '/content/drive/MyDrive/fdl21-fdl-dsba/best_model_psnet3.pt'
device = torch.device('cpu')
model = torch.load(PATH, map_location=torch.device('cpu'))
model.eval();

In [ ]:
print(model)

# Test set loading and submission file

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

class TestDataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.name = os.listdir(os.path.join(path, 'test_images'))

    def __len__(self):
        return len(self.name)

    def __getitem__(self, index):
        img_name = self.name[index]
        img_path = os.path.join(self.path, 'test_images',img_name)
        assert os.path.isfile(img_path)
        img = Image.open(img_path)
        img = img.resize((512,512))
        img = transform(img)
        
        return img

In [ ]:
test_dataset = TestDataset('/content/drive/MyDrive/fdl21-fdl-dsba/test_images')
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False)
it = next(iter(test_loader))
print(it.shape,it.dtype)

torch.Size([2, 3, 512, 512]) torch.float32


In [ ]:
from tqdm.auto import tqdm as tq
test_mask=[]
for data in tq(test_loader):
    output = model(data).cpu().detach().numpy()
    print(output.shape)
    for b in range(output.shape[0]):
                   test_mask.append(output[b])


  0%|          | 0/56 [00:00<?, ?it/s]

(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 512, 512)
(2, 27, 51

In [ ]:
mask[0].shape

(512, 512)

In [ ]:
tarr = []
for x in os.listdir('test_images'):
    imp = os.path.join('test_images',x)
    img = Image.open(imp)
    i = np.array(img)
    tarr.append(i)

In [ ]:
path1 = '/content/drive/MyDrive/fdl21-fdl-dsba/test_masks3'
path2 = '/content/drive/MyDrive/fdl21-fdl-dsba/test_images/test_images'
file_name = list(os.listdir(path2))
for i in range(len(mask)):
    mask_name = file_name[i]
    mask_name = mask_name.replace('jpg','png')
    file_path = os.path.join(path1,mask_name)
    img = Image.fromarray(mask[i].astype('uint8'))
    img.save(file_path)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


In [ ]:
import os

from PIL import Image
import numpy as np

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


def create_rles():
    """Used for Kaggle submission: predicts and encode all test images"""
    dir = '/content/drive/MyDrive/fdl21-fdl-dsba/test_masks3/'
    N = len(list(os.listdir(dir)))
    with open('submission_file_pspnet3.csv', 'w') as f:
        f.write('ImageClassId,rle_mask\n')
        for index, i in enumerate(os.listdir(dir)):
            # print('{}/{}'.format(index, N))

            mask = Image.open(dir+i)
            mask = mask.resize((1024, 1024), resample=Image.NEAREST)
            mask = np.array(mask)

            for x in range(1, 25):
                enc = rle_encode(mask == x)
                f.write(f"{i.split('_')[0]}_{x},{enc}\n")

create_rles()